In [1]:
import os
os.chdir("/mmsegmentation")
os.getcwd()

'/mmsegmentation'

In [2]:
import torch
import numpy as np
from mmengine.config import Config
from mmseg.models import build_segmentor
from mmengine.runner import load_checkpoint
from mmseg.datasets import CityscapesDataset
from torch.utils.data import DataLoader
from mmcv.transforms import Compose
from tqdm import tqdm
from mmengine.registry import init_default_scope
from mmengine.dataset import default_collate
from mmseg.evaluation import IoUMetric
from mmengine.structures import PixelData
from mmseg.structures import SegDataSample

from mmseg.apis import init_model, inference_model
from torchvision.datasets import Cityscapes
import evaluate
from mmengine.dataset import default_collate
from mmengine.logging import HistoryBuffer

from function import *
from evaluation import *
from dataset import CitySet, ADESet

from pixle import Pixle

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cf_path = './configs/segformer/segformer_mit-b5_8xb1-160k_cityscapes-1024x1024.py'
ckpt_path = './checkpoint/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth'

# 2. Default scope 초기화
model = init_model(cf_path, ckpt_path, 'cuda')

mean_iou = evaluate.load("mean_iou", "segmentation")

dataset = CitySet('./datasets/cityscapes')

/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:250: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by local backend from path: ./checkpoint/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth


In [5]:
img_list = []
pred_list = []
gt_list = []

# pixle = Pixle(model)

with torch.no_grad():
    for img, _, gt in tqdm(dataset):
        # print(batch['data_samples'][0].gt_sem_seg.data.max())
        img = img[:,:,::-1]
        
        result = inference_model(model, img)
        result = result.pred_sem_seg.data.squeeze().cpu().numpy().astype(np.uint8) #result shape (1024, 2048)
        img_list.append(img)
        pred_list.append(result)
        gt_list.append(gt)
        # print(result.pred_sem_seg.data.squeeze().shape)

    iou = mean_iou.compute(
        predictions=pred_list,
        references=gt_list,
        num_labels=19,
        ignore_index=255,
        reduce_labels=False,
    ) 
    print(iou['mean_iou'])

100%|██████████| 100/100 [00:36<00:00,  2.72it/s]


0.773551937869543


In [7]:
pred_list = []
gt_list = []
# 원본 이미지가 필요하다면: img_list_all = []

num_samples = len(dataset)
batch_size = 5 # 배치 크기 설정

# 만약 MMSegInferencer를 사용한다면 여기서 초기화
# inferencer = MMSegInferencer(model=model)

with torch.no_grad():
    # 전체 샘플 수를 배치 크기 단위로 순회
    for i in tqdm(range(0, num_samples, batch_size), desc="배치 단위 추론 중"):
        # 현재 배치에 해당하는 데이터 인덱스
        batch_indices = range(i, min(i + batch_size, num_samples))
        # 인덱스가 없으면 (마지막 부분 처리 후) 건너뛰기
        if not batch_indices:
            continue

        # 배치 데이터 준비 (튜플의 리스트)
        batch_data = [dataset[j] for j in batch_indices]
        # 배치 데이터가 비어있으면 건너뛰기
        if not batch_data:
            continue

        # NumPy 이미지 배열 리스트 생성
        # sample[0]이 이미지, sample[2]가 GT라고 가정
        try:
            img_batch_list = [sample[0] for sample in batch_data]
            gt_batch_list = [sample[2] for sample in batch_data]
        except IndexError:
            print(f"오류: 배치 인덱스 {i} 근처에서 데이터 형식이 올바르지 않습니다. 샘플이 (이미지, ?, GT) 형태인지 확인하세요.")
            continue # 이 배치는 건너뛰기

        # 입력 이미지 리스트가 비었는지 확인
        if not img_batch_list:
            continue

        try:
            # inference_model에 NumPy 배열 리스트 직접 전달
            # 이 함수가 리스트 입력을 처리하고, 결과 리스트를 반환한다고 가정
            results = inference_model(model, img_batch_list)

            # 또는 최신 Inferencer API 사용 시:
            # results = inferencer(img_batch_list, return_datasample=True) # 결과 형식이 다를 수 있음

            # 결과 처리 (results가 SegDataSample 객체의 리스트라고 가정)
            current_batch_preds = []
            for result_sample in results:
                # 결과 객체 구조에 따라 pred_sem_seg 접근
                # 예: MMSegInferencer 사용 시 result_sample['predictions'][0] 와 같은 형태일 수 있음
                pred_map = result_sample.pred_sem_seg.data.squeeze().cpu().numpy().astype(np.uint8)
                current_batch_preds.append(pred_map)

            # 전체 결과 리스트에 현재 배치 결과 추가
            pred_list.extend(current_batch_preds)
            gt_list.extend(gt_batch_list) # 해당 배치의 GT들도 추가
            # 원본 이미지가 필요하다면: img_list_all.extend(img_batch_list)

        except Exception as e:
            print(f"오류 발생 (배치 시작 인덱스: {i}): {e}")
            # 필요하다면 오류 발생 시 해당 배치를 건너뛰거나 다른 처리 수행
            print("이 배치를 건너<0xEB><0x9B><0x84>니다.")


# 루프 종료 후 IoU 계산
if pred_list and gt_list:
    # 계산 전 예측과 정답 리스트 길이 확인
    if len(pred_list) != len(gt_list):
         print(f"경고: 예측 결과 수({len(pred_list)})와 정답 레이블 수({len(gt_list)})가 일치하지 않습니다!")
    else:
        iou = mean_iou.compute(
            predictions=pred_list, # NumPy 배열 리스트
            references=gt_list,  # NumPy 배열 리스트
            num_labels=19,
            ignore_index=255,
            reduce_labels=False,
        )
        print(f"Mean IoU: {iou['mean_iou']}")
else:
    print("처리된 결과가 없어 IoU를 계산할 수 없습니다.")

배치 단위 추론 중: 100%|██████████| 20/20 [00:34<00:00,  1.72s/it]


Mean IoU: 0.8023886453206299


In [7]:
visualize_segmentation(img_list[0], gt_list[0], save_path='/mmsegmentation/results/pixle_benign1.png', alpha=0.5)

In [19]:
img, filepath, gt = dataset[0:2]

In [23]:
print(img)
print(gt.shape)

[array([[[ 82,  63,   0],
        [ 79,  62,   0],
        [ 76,  60,   0],
        ...,
        [187, 184, 155],
        [189, 185, 156],
        [198, 193, 148]],

       [[ 63, 104,  89],
        [ 64, 102,  86],
        [ 65, 100,  84],
        ...,
        [186, 183, 154],
        [187, 183, 155],
        [197, 192, 147]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [185, 182, 153],
        [186, 182, 154],
        [196, 191, 146]],

       ...,

       [[158, 176, 160],
        [156, 175, 160],
        [161, 177, 164],
        ...,
        [143, 149, 121],
        [142, 148, 120],
        [141, 147, 119]],

       [[158, 176, 160],
        [156, 175, 160],
        [161, 177, 164],
        ...,
        [ 35,  38,  31],
        [143, 164, 141],
        [143, 163, 140]],

       [[158, 176, 160],
        [156, 175, 161],
        [161, 177, 164],
        ...,
        [ 39,  43,  34],
        [ 37,  40,  32],
        [ 35,  38,  30]

AttributeError: 'list' object has no attribute 'shape'